In [2]:
import requests

In [3]:
import numpy as np

In [4]:
from bs4 import BeautifulSoup

In [5]:
from pandas import DataFrame

In [6]:
url = "https://www.data.jma.go.jp/obd/stats/etrn/upper/view/daily_uth.php?year=1999&month=5&day=7&hour=9&atm=&point=47918&view="

In [7]:
req = requests.get(url)

In [8]:
soup = BeautifulSoup(req.content, "html.parser")

In [9]:
table1 = soup.find('table', id='tablefix1')

In [10]:
th_all = table1.find_all('th')

In [11]:
print(th_all)

[<th scope="col">気圧(hPa)</th>, <th scope="col">ジオポテンシャル<br/>高度(m)</th>, <th scope="col">気温(℃)</th>, <th scope="col">相対湿度(%)</th>, <th scope="col">識別符</th>]


In [12]:
table1_column = []

In [13]:
for th in th_all:
    table1_column.append(th.string)

In [14]:
print(table1_column)

['気圧(hPa)', None, '気温(℃)', '相対湿度(%)', '識別符']


Noneと表示されてしまった。  
理由としては  
```the_all```のジオポテンシャルと高度(m)の間に  
`<br>`が入ってしまっているためである。  
解決策として、`.string`の代わりに`.text`を使う。

In [15]:
table1_column = []

In [16]:
for th in th_all:
    table1_column.append(th.text)

In [17]:
print(table1_column)

['気圧(hPa)', 'ジオポテンシャル高度(m)', '気温(℃)', '相対湿度(%)', '識別符']


In [18]:
tr_all = table1.find_all('tr')

In [19]:
tr_all = tr_all[1:]

In [20]:
col_num = len(table1_column)

In [21]:
row_num = len(tr_all)

In [22]:
table1_data = np.zeros((row_num, col_num), dtype=object)

`dtype=object`*重要*  
NumPy配列には同じ型の要素しか格納できない  
`dtype`を指定しないと気圧や気温は数値型であるのに対し、  
識別符の要素は文字列のためnumpy配列に要素を格納するときにエラーがでる

In [23]:
for r, tr in enumerate(tr_all):
    td_all = tr.find_all('td')
    table1_data[r,:] = [td.string for td in td_all]

In [24]:
df = DataFrame(data=table1_data, columns=table1_column)

In [25]:
display(df.head())

,気圧(hPa),ジオポテンシャル高度(m),気温(℃),相対湿度(%),識別符
0,1012.0,7,18.7,87,特異点
1,1002.1,91,18.0,78,None
2,977.9,301,17.4,82,None
3,848.1,1505,10.7,95,特異点
4,764.3,2369,7.6,95,None


In [26]:
display(df.tail())

,気圧(hPa),ジオポテンシャル高度(m),気温(℃),相対湿度(%),識別符
47,55.3,19962,-69.8,///,特異点
48,51.9,20345,-64.1,///,None
49,49.3,20661,-61.7,///,特異点
50,45.0,21229,-59.7,///,None
51,42.9,21527,-59.8,///,特異点


In [29]:
title = soup.find('h3')
print(title.text)

石垣島　1999年5月7日9時


In [31]:
df.to_csv("./ishigaki19990507.csv")